In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from extract import LOCM3
from utils import read_plan, read_json_file
from evaluator import ExecutabilityEvaluator
from collections import defaultdict
from itertools import combinations, permutations

In [2]:
def test_acceptance(domain_name,train_traces, test_traces):
    debug = {}
    try:
        locm = LOCM3( debug=debug)
        model = locm.extract_model(train_traces)
        learned_domain = model.to_pddl_domain(domain_name)

        evaluator = ExecutabilityEvaluator(learned_domain, debug=True)
        exe = evaluator.get_acceptance_rate(test_traces, test_traces)

        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0,1
    
def test_self_acceptance(domain_name,train_trace):
    debug = {}
    try:
        locm = LOCM3(event_arity=2, debug=debug)
        model = locm.extract_model([train_trace])
        learned_domain = model.to_pddl_domain(domain_name)

        evaluator = ExecutabilityEvaluator(learned_domain, debug=True)
        exe = evaluator.get_acceptance_rate(train_trace)

        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0,1

In [3]:
def generate_pos_tuples(pos, arity: int):
    result = []
    k = min(len(pos), arity)
    for r in range(1, k+1):
        for comb in combinations(pos, r):
            for perm in permutations(comb):
                result.append(tuple(perm))
    return result

def generate_sort_tuples(pos, arity: int):
    result = []
    k = min(len(pos), arity)
    for r in range(1, k+1):
        for comb in combinations(pos, r):
            result.append(tuple(comb))
                
    return result

In [4]:
res = generate_pos_tuples([1, 2, 3], 3)
print(res)

res2 = generate_sort_tuples([1, 1], 2)
print(res2)

[(1,), (2,), (3,), (1, 2), (2, 1), (1, 3), (3, 1), (2, 3), (3, 2), (1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]
[(1,), (1,), (1, 1)]


In [5]:
# test_plan = "(pickup b2?object),(stack b2?object b1?object),(unstack b2?object b1?object),(putdown b2?object),(pickup b3?object),(stack b3?object b2?object)"
# test_trace = read_plan(test_plan)


# # test_plan2 = "(pickup b2?object),(stack b2?object b1?object),(unstack b3?object b4?object),(putdown b3?object)"
# # test_trace2 = read_plan(test_plan)

# debug = {
#     'trace_to_obj_trace' : True
# }
# l3 = LOCM3(event_arity=2, debug=debug, viz=True)
# model = l3.extract_model([test_trace])


In [6]:
# pddl_domain = model.to_pddl_domain("test_domain")
# pddl_domain.dump()
# evaluator = ExecutabilityEvaluator(pddl_domain, debug=True)
# exe = evaluator.get_acceptance_rate(test_trace, None)
# print(exe)

In [7]:
plain_traces = defaultdict(lambda: defaultdict())
with open("../../data/plain_traces/plain_traces.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        details = line.split("&&")

        domain_name = details[0]
        problem_name = details[1]
        plan = details[-1]

        plain_traces[domain_name][problem_name]= read_plan(plan)

In [8]:
with open("test_locm3_perm.csv", "w") as f:
    for domain in plain_traces:
        for prob in plain_traces[domain]:
            try:
                trace = plain_traces[domain][prob]
                acceptance_rate = test_self_acceptance(domain, trace)
                acc, _acc = acceptance_rate
                if acc <1:
                    print(f"Domain: {domain}, Problem: {prob}, Acceptance Rate: {acc}")
                f.write(f"{domain},{acceptance_rate[0]},{acceptance_rate[1]}\n")
            except Exception as e:
                print(f"Error processing trace in domain {domain}: {e}")
                f.write(f"{domain},0,1\n")

action <PropositionalAction '(push zero loc_4_4 loc_4_3 loc_4_2 left box1)' at 0x73a194b12060> not executable
preconditions not satisfied:  {<Atom m-push.(4,)-s1(left)>, <Atom m-push.(0,)-s1(zero)>}
Domain: sokoban, Problem: p04.pddl, Acceptance Rate: 0
action <PropositionalAction '(move zero loc_6_5 loc_5_5 up)' at 0x73a194b83440> not executable
preconditions not satisfied:  {<Atom m-move.(3,)-s1(up)>}
Domain: sokoban, Problem: p27.pddl, Acceptance Rate: 0
action <PropositionalAction '(push zero loc_4_4 loc_5_4 loc_6_4 down box1)' at 0x73a194b861b0> not executable
preconditions not satisfied:  {<Atom m-push.(4,)-s1(down)>, <Atom m-push.(0,)-s1(zero)>}
Domain: sokoban, Problem: p02.pddl, Acceptance Rate: 0
action <PropositionalAction '(push zero loc_4_4 loc_3_4 loc_2_4 up box1)' at 0x73a194b83b60> not executable
preconditions not satisfied:  {<Atom m-push.(4,)-s1(up)>, <Atom m-push.(0,)-s1(zero)>}
Domain: sokoban, Problem: p01.pddl, Acceptance Rate: 0
action <PropositionalAction '(push